## Installing Required Packages

In [ ]:
# !pip install web3

 ### gas price:4 gwei

In [ ]:
# !pip install --force-reinstall jsonschema==3.2.0
# exit()

In [ ]:
!pip install py-solc-x

In [ ]:
import time
import json
from web3 import Web3
from solcx import compile_standard, install_solc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Connecting to Test Network : Tunneled Ganache

In [ ]:
# Connecting to local instance of ganche tunneled using ngrok
# Run a local instance of ganache-cli on ubuntu (current run env) and tunnel the http port (ngrok http 8545, for ganache)
# After tunneling the local blockahin network (ganache) should be accessible online (Google Colab Notebook)
web3 = Web3(Web3.HTTPProvider("https://1c97-192-214-232-26.ngrok-free.app"))
web3.is_connected()

True

In [ ]:
with open("/content/drive/MyDrive/Smart Contracts/Main.sol", "r") as file:
    contract = file.read()

## Installing the correct version of Solidity

In [ ]:
solc_version = '0.8.12'
install_solc(solc_version)

Version('0.8.2')

In [ ]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"Main.sol": {"content": contract}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=solc_version,
)

print('Smart contract is compiled successfully', compiled_sol)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

Smart contract is compiled successfully {'contracts': {'Main.sol': {'Main': {'abi': [{'inputs': [], 'name': 'retrieve', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': 'num', 'type': 'uint256'}], 'name': 'store', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}], 'evm': {'bytecode': {'generatedSources': [], 'linkReferences': {}, 'object': '608060405234801561001057600080fd5b5061012f806100206000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c80632e64cec11460375780636057361d146051575b600080fd5b603d6069565b6040516048919060c2565b60405180910390f35b6067600480360381019060639190608f565b6072565b005b60008054905090565b8060008190555050565b60008135905060898160e5565b92915050565b60006020828403121560a057600080fd5b600060ac84828501607c565b91505092915050565b60bc8160db565b82525050565b600060208201905060d5600083018460b5565b92915050565b60008190509190

In [ ]:
abi = json.loads(compiled_sol["contracts"]["Main.sol"]["Main"]["metadata"])["output"]["abi"]
bytecode = compiled_sol["contracts"]["Main.sol"]["Main"]["evm"]["bytecode"]["object"]

## Deploying contract on ganche local network

Try to deploy the smart contract on ganache using https://coinsbench.com/how-to-deploy-and-interact-with-solidity-contracts-with-python-and-ganache-be63334323e6


In [ ]:
def sendeth(address_from, private_key, address_to, val):
  nonce = web3.eth.get_transaction_count(address_from)
  tx = {
      'gas': 2000000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
      'to': address_to,
      'value': web3.to_wei(str(val), 'ether'),
  }
  signed_txn = web3.eth.account.sign_transaction(tx, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1

In [ ]:
# Make sure to run ganache with -d flag to create the same accounts deterministically.

ganache_pk = [
    '0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d',
    '0x6cbed15c793ce57650b9877cf6fa156fbef513c4e6134f022a85b1ffdd59b2a1',
    '0x6370fd033278c143179d81c5526140625662b8daa446c22ee2d73db3707e620c',
    '0x646f1ce2fdad0e6deeeb5c7e8e5543bdde65e86029e2fd9fc169899c440a7913',
    '0xadd53f9a7e588d003326d1cbf9e4a43c061aadd9bc938c843a79e7b4fd2ad743',
    '0x395df67f0c2d2d9fe1ad08d1bc8b6627011959b79c53d7dd6a3536a33ab8a4fd',
    '0xe485d098507f54e7733a205420dfddbe58db035fa577fc294ebd14db90767a52',
    '0xa453611d9419d0e56f499079478fd72c37b251a94bfde4d19872c44cf65386e3',
    '0x829e924fdf021ba3dbbc4225edfece9aca04b929d6e75613329ca6f1d31c0bb4',
    '0xb0057716d5917badaf911b193b12b910811c1497b5bada8d7711f758981c3773'
]

In [ ]:
# Account details from which contract will be deployed
deployment_account = web3.eth.account.create()

deployment_account_add = deployment_account.address
deployment_account_pk = deployment_account.key.hex()
# Sending 90 ETH to the deployer's account
sendeth(web3.eth.accounts[0], ganache_pk[0], deployment_account_add, 90)
print(web3.eth.get_balance(deployment_account_add))

# Create the contract instance
Contract = web3.eth.contract(abi=abi, bytecode=bytecode)

# Get the number of latest transaction
nonce = web3.eth.get_transaction_count(deployment_account_add)

# build transaction
transaction = Contract.constructor().build_transaction(
    {
        "chainId": web3.eth.chain_id,
        "gasPrice": web3.eth.gas_price,
        'gas': 2000000,
        # 'gasPrice': web3.to_wei('4', 'gwei'),
        "from": deployment_account_add,
        "nonce": nonce,
    }
)

# Sign the transaction
sign_transaction = web3.eth.account.sign_transaction(transaction, private_key=deployment_account_pk)
print("Deploying Contract!")
# Send the transaction
transaction_hash = web3.eth.send_raw_transaction(sign_transaction.rawTransaction)
# Wait for the transaction to be mined, and get the transaction receipt
print("Waiting for transaction to finish...")
transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash)
contract_address = transaction_receipt.contractAddress
print(f"Done! Contract deployed to {contract_address}")

AttributeDict({'transactionHash': HexBytes('0xcfd96d3f318d95e8dbfe294d0cdd54ae8332395998d5992c9b61c20570b9a416'), 'transactionIndex': 0, 'blockHash': HexBytes('0xb2926238127dd2c8a622bef9bb5bb7ef81887b0596b4993d91de056d5699b6bb'), 'blockNumber': 1, 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1', 'to': '0x2c0FF54b50A2B4834D1d7cD5282d3BC81A9bb5D9', 'gasUsed': 21000, 'cumulativeGasUsed': 21000, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})
90000000

In [ ]:
contract = web3.eth.contract(address = contract_address, abi = abi)

## Client Accounts

In [ ]:
acc_add = []
acc_pk = []
NUM_ACCOUNTS = 100
acc_index = 1
for i in range(NUM_ACCOUNTS):
  print('Account',i)
  acc = web3.eth.account.create()
  acc_add.append(acc.address)
  acc_pk.append(acc.key.hex())
  try:
    sendeth(web3.eth.accounts[acc_index], ganache_pk[acc_index], acc.address, 1)
  except:
    acc_index += 1
    sendeth(web3.eth.accounts[acc_index], ganache_pk[acc_index], acc.address, 1)


## Functions from smart contract

In [ ]:
def make_me_client(account_address, private_key):
  nonce = web3.eth.get_transaction_count(account_address)
  buildData = {
      'from': account_address,
      'chainId': web3.eth.chain_id,
      'gas': 2000000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
  }
  txn_dict = contract.functions.makemeclient().build_transaction(buildData)
  signed_txn = web3.eth.account.sign_transaction(txn_dict, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1

In [ ]:
# for now modelId = 0
def data_post(account_address, private_key, clientData, modelId=0):
  nonce = web3.eth.get_transaction_count(account_address)
  buildData = {
      'from': account_address,
      'chainId': web3.eth.chain_id,
      'gas': 200000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
  }
  txn_dict = contract.functions.data_post(modelId, clientData).build_transaction(buildData)
  signed_txn = web3.eth.account.sign_transaction(txn_dict, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1

In [ ]:
def model_aggregation(account_address, private_key, modelId=0):
  nonce = web3.eth.get_transaction_count(account_address)
  buildData = {
      'from': account_address,
      'chainId': web3.eth.chain_id,
      'gas': 200000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
  }
  txn_dict = contract.functions.model_aggregation(modelId).build_transaction(buildData)
  signed_txn = web3.eth.account.sign_transaction(txn_dict, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1


In [ ]:
def read_index(account_address, private_key, clientIndex=0, modelId=0):
  nonce = web3.eth.get_transaction_count(account_address)
  buildData = {
      'from': account_address,
      'chainId': web3.eth.chain_id,
      'gas': 200000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
  }
  txn_dict = contract.functions.readindex(clientIndex, modelId).build_transaction(buildData)
  signed_txn = web3.eth.account.sign_transaction(txn_dict, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1

In [ ]:
def data_request(account_address, private_key, clientIndex=0, modelId=0):
  nonce = web3.eth.get_transaction_count(account_address)
  buildData = {
      'from': account_address,
      'chainId': web3.eth.chain_id,
      'gas': 200000,
      'gasPrice': web3.eth.gas_price,
      'nonce': nonce,
      'value': web3.to_wei('17', 'gwei'),
  }
  txn_dict = contract.functions.data_request().build_transaction(buildData)
  signed_txn = web3.eth.account.sign_transaction(txn_dict, private_key)
  tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
  Loop = 0
  while(Loop < 10):
    try:
      tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
      print(tx_receipt)
      break
    except:
      print('Waiting')
      time.sleep(10)
      Loop += 1

In [ ]:
# web3 = Web3(Web3.HTTPProvider("https://sepolia.infura.io/v3/5ba2715a3ef74bfc8da221c19851574d"))

# !pip install eth_account
# from eth_account import Account
# NUM_ACCOUNTS = 5
# acc_add = []
# acc_pk = []
# for i in range(NUM_ACCOUNTS):
#   acct = Account.create(str(i))
#   acc_add.append(acct.address)
#   acc_pk.append(acct.key)

# web3.eth.get_balance('0x984C96861ED9055e9ce32cB4aB898F6aE0C09ACd')

# contract_address = '0xBb07932603A931293d4a89e278CE2Af4FD78c553'
# # First 4 accounts (clients) are acting as trainers,
# # Next 2 are level 1 aggregators, last is level 2 aggregator (2 level Hierarchal Aggregation)
# account_addresses = ['0x09750F3e65bBA855AA69cD33877015b02E4Ac16E',
#                      '0xF6EF9953675cfF6C9ebd98e8f9B9c58Eb695e74F',
#                      '0x8F72170d5601E8cE8f731B7da4C72Abc626Bb2e2',
#                      '0xec4D250654042A372f6e779e1d325A6754D74bb6',
#                      '0xf0AafaAC8b6D258dEE872b1bE8E30283152e420B',
#                      '0x8A3057cCD0602ba4Fd125eAf97c9d4E21974D613',
#                      '0x8d6309Be92dC1513F41E922C5c42bdE91edDB7B3'
#                      ]
# private_keys = ['f06c206f2e21d1f5a6299e3f676dbb2cd26f8ffa993addaa8cc51d50a2798150',
#                 'c083222f68780a4c2777ec7e983e330ab171ae64ff07c711dca0a77709608fbc',
#                 '5179b171ad691f6d1af4f5e4ee0d660fd5268e6d6792225a80ef61437082caaa',
#                 '41443d9d841927fa58c11e529cfc8085fd9739fe52a3d3ec4239693c48ca6d45',
#                 'c4a89b349d3ace91cc45e4fc8bf17562721d9633032b6124df5f3ddc4d1aa687',
#                 '2c407847a0bce1b89bb5073b937e73078b5fb7b8692e2bdaf51ab3d80417f1a5',
#                 'c84c8acc342a8f54fa952af8e1142e2b55972071d7c7c705aadf7033d9fc97c1'
#                 ]


## REFERANCES

1. https://medium.com/@mweiss.eth/send-ether-with-web3-py-python-bd0e8e85e93e (Sending Eth between accounts)
2. https://ethereum.stackexchange.com/questions/69831/ganache-cli-how-to-persist-new-created-accounts (Deterministically setting up Ganache local network)
3. https://www.alchemy.com/gwei-calculator (Currency Conversions)
4. https://coinsbench.com/how-to-deploy-and-interact-with-solidity-contracts-with-python-and-ganache-be63334323e6 (Deploying SC via Google Colab on local ganche network)
5. https://web3js.readthedocs.io/en/v1.2.11/web3-eth-accounts.html (Web3 API)
6. https://www.youtube.com/watch?v=DCxt9SAnkyc (Setting up ngrok)
7. https://web3py.readthedocs.io/en/stable/transactions.html (Web3 API Transaction related docs)